<a href="https://colab.research.google.com/github/tamburins/ESAA-2022-/blob/main/ESAA_5_5(512~516%2C529~543).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 06 토픽 모델링(topic modeling) - 20 뉴스그룹

문서 집합에 숨어있는 주제를 찾아내는 것으로 더 함축적인 의미로 문장을 요약하는 사람의 토픽모델링에 비해 머신러닝 기반의 토픽모델은 숨겨진 주제를 효과적으로 표현할 수 있는 중심 단어를 함축적으로 추출합니다.

머신러닝 기반의 토픽모델링에 자주 사용되는 기법은 lsa와 lda로 이 절에서는 lda만을 이용해 토픽 모델링을 수행하겠다. 이는 차원축소의 lda와는 다른 알고리즘이다. 토픽모델링은 텍스트분류에서 소개한 20뉴스그룹 데이터세트를 이용해 적용하고자 한다.

여러 주제중 모토사이클, 야구, 그래픽스, 윈도우, 중동, 기독교, 전자공학, 의학의 8개 주제를 추출하고 이들 텍스트에 lda 기반의 토픽 모델링을 적용해보고자한다.

먼저 lda 토픽 모델링을 위해 fetch_20newsgroups()는 categories 파라미터를 통해 필요한 주제만 필터링해 추출하고 추출된 텍스트를 count 기반으로 벡터화 변환한다. lda는 count 기반의 벡터화만 사용하며 max_ features1000통해 word 피처의 개수를 제한하고 ngram_range는 (1,2)로 설정하여 피처벡터화 변환하겠다

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# 모토사이클, 야구, 그래픽스, 윈도우즈 중동 기독교 전자공학 의학 8개 주제 추출
cats = ['rec.motorcycles', 'rec.sport.baseball', 'comp.graphics', 'comp.windows.x','talk.politics.mideast', 'soc.religion.christian', 'sci.electronics', 'sci.med']
# 위에서 cats 변수로 기재된 카테고리만 추출하고 fetch_20groups에 cats 입력
news_df = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'),
                            categories=cats, random_state=0)
#lda is adapted in count based vectoreization
count_vect = CountVectorizer(max_df=0.95, max_features=1000, min_df=2, stop_words='english',
                             ngram_range=(1,2))
fect_vect=count_vect.fit_transform(news_df.data)
print('CountVectorizer shape: ', fect_vect.shape)

CountVectorizer shape:  (7862, 1000)


카운트 벡터라이저 객체변수인 featvet모두 7000여개의 문서가 1000여개의 피처로 구성된 행렬데이터이다. 이렇게 벡터화된 데이터세트를 기반을 lda 토픽 모델링을 수행한다. 토픽의 개수는 cats 개수인 8개로 Latentdirichletallocation 클래스의 ncomponents파라미터를 이용해 토픽개수를 조정한다.

In [ ]:
lda = LatentDirichletAllocation(n_components=8, random_state=0)
lda.fit(fect_vect)

LatentDirichletAllocation(n_components=8, random_state=0)

lda.fit을 수행하면 객체는 component 속성값을 갖는데 이는 개별 토픽별로 word 피처가 얼마나 많은지 그 토픽에 할당됐는지에 대한 수치를 갖는다. 높은 갑일 수록 해당 word피처는 토픽의 중심 word가 된다.

In [ ]:
print(lda.components_.shape)
lda.components_

(8, 1000)


array([[3.60992018e+01, 1.35626798e+02, 2.15751867e+01, ...,
        3.02911688e+01, 8.66830093e+01, 6.79285199e+01],
       [1.25199920e-01, 1.44401815e+01, 1.25045596e-01, ...,
        1.81506995e+02, 1.25097844e-01, 9.39593286e+01],
       [3.34762663e+02, 1.25176265e-01, 1.46743299e+02, ...,
        1.25105772e-01, 3.63689741e+01, 1.25025218e-01],
       ...,
       [3.60204965e+01, 2.08640688e+01, 4.29606813e+00, ...,
        1.45056650e+01, 8.33854413e+00, 1.55690009e+01],
       [1.25128711e-01, 1.25247756e-01, 1.25005143e-01, ...,
        9.17278769e+01, 1.25177668e-01, 3.74575887e+01],
       [5.49258690e+01, 4.47009532e+00, 9.88524814e+00, ...,
        4.87048440e+01, 1.25034678e-01, 1.25074632e-01]])

componetns는 8행 4000열로 구성되어있는데, 이는 8개의 토픽별로 1000개의 피ㅓ가 해당 토픽별로 연관도 값을 갖는 것을 의미한다. 이러한 components 값만으로 는 각 토픽별 word 연관도를 보기가 어려우니 display topics funnction을 생성하여 토픽별 연관도가 높은 순으로 word를 나열해보고자 함

In [ ]:
def display_topics(model, feature_names, no_top_words):
  for topic_index, topic in enumerate(model.components_):
    print('Topic #', topic_index)

    #coponentsarray에서 가장 값이 큰 순으로 정ㄹ려시 그값의 array인덱스 반환
    topic_word_indexes = topic.argsort()[::-1]
    topic_indexes = topic_word_indexes[:no_top_words]

    #top indexes 대상인 인덱스별로 feature names에 해당하는 word feature 추출 후 join으로 concat
    feature_concat = ''.join([feature_names[i] for i in topic_indexes])
    print(feature_concat)

# in countvectorizer extract names of all word by get_feature_names()
feature_names = count_vect.get_feature_names_out()
#메소드 이름 변경(?)

#top 15 of 연관도 for each topic
display_topics(lda, feature_names,15)


Topic # 0
year10gamemedicalhealthteam1220diseasecancer1993gamesyearspatientsgood
Topic # 1
donjustlikeknowpeoplesaidthinktimevedidnrightgoingsayllway
Topic # 2
imagefilejpegprogramgifimagesoutputformatfilescolorentry00usebit03
Topic # 3
likeknowdonthinkusedoesjustgoodtimebookreadinformationpeopleusedpost
Topic # 4
armenianisraelarmeniansjewsturkishpeopleisraelijewishgovernmentwardos dosturkeyarabarmenia000
Topic # 5
educomavailablegraphicsftpdatapubmotifmailwidgetsoftwaremitinformationversionsun
Topic # 6
godpeoplejesuschurchbelievechristdoeschristiansaythinkchristiansbiblefaithsinlife
Topic # 7
usedosthankswindowsusingwindowdoesdisplayhelplikeproblemserverneedknowrun


topic 0의 경우 명확하지 않고 일반적인 단어가 주를 이루며 1의 경우 명확하게 컴퓨터그래픽스 단어가 추출되었다. 2는 기독교 관련 주제어가, 3은 의학에 관한 주제어가, 4는 윈도우 운영체제와 관련한 주제어가 추출되었으나 5는 일반적인 단어로 주제어가 추출되었다. 6은 중동분쟁관련어 7은 애매하지만 우니도우 운영체제와 관한 주제어가 일부 추출되었다. 0,5,7의 토픽에서 애매한 주제어가 추출되었다.

## 08 문서 유사도
( 528~

### 문서 유사도 측정 방법 - 코사인 유사도
이는 벡터와 벡터간 유사도를 비교할 때 벡터의 크기보다는 벡터의 상호방향성이 얼마나 유사한지에 기반으로 한다.

### 두 벡터 사잇각
두 벡터의 사잇각에 따라서 상호관계는 유사하거나 관련이 없거나 아예 반대관계가 될 수 있다. 두 벡터사이 cos값은 두 벡터 a,b 의 내적이 a의 길이와 b의 길이의 크기와 두 벡터사이 코사인값으로 나타내는 것에서 기인하여, 유사도 cos을 구할 있다. 유사도는 두 벡터의 내적을 총 벡터크기의 합으로 나눈 것, 즉 내적 결과를 총 벡터의 크기로 정규화한 것이라 할 수 있다.

문서를 피처벡터화 변환시 많은 희소행렬이 되기 쉬운데 이러한 희소행렬 기반에서 문서와 문서벡터간의 크기에 기반한 유사도지표는 정확도가 떨어지며, 문서가 매우 긴 경우 단어의 빈도수가 또한 많기 때문에 빈도수 기반도 어렵다

간단한 문서에 대해 서로간의 문서 유사도를 코사인 유사도 기반으로 구해보자.

In [ ]:
import numpy as np
def cos_similarity(v1, v2):
  dot_product = np.dot(v1,v2)
  _norm = (np.sqrt(sum(np.square(v1)))*np.sqrt(sum(np.square(v2))))
  similarity = dot_product / _norm

  return similarity

In [ ]:
# tf idf 변환
from sklearn.feature_extraction.text import TfidfVectorizer
doc_list = ['if you take the blue pill, the story ends',
'if you take take red pill, you stay in Wonderland',
'if you take the red pill, I show you how dep the rabbit hole goes']

tfidf_vect_simple = TfidfVectorizer()
feature_vect_simple = tfidf_vect_simple.fit_transform(doc_list)
print(feature_vect_simple.shape)


(3, 18)


변환된 행렬은 희소행렬로 앞에서 작성한 cos similarity 함수의 인자인 array로 만들기 ㅜ이해 밀집행렬로 변환한 뒤 다시 각각을 배열로 반환한다. feature_vect_dense[0]은 doc list 첫번/재 문서의 피처벡터화, feature_vect_dense[1]은 doc list 2번재 문서의 피처벡터화이다.

In [ ]:
#with tfidf vectorizer, transform result is needed to transformation to dense
feature_vect_dense = feature_vect_simple.todense()

# firest sentence and second sentence's extraction of feature
vect1 = np.array(feature_vect_dense[0]).reshape(-1,)
vect2 = np.array(feature_vect_dense[1]).reshape(-1,)

# with feature vector of first n second sentence, extract cos similarity
similarity_simple = cos_similarity(vect1, vect2)
print('sentence1, sentence2 cosine similarity: {:.3f}'.format(similarity_simple))

sentence1, sentence2 cosine similarity: 0.304


In [ ]:
#get similarity with sent2,3 and sent1,3

vect1 = np.array(feature_vect_dense[0]).reshape(-1,)
vect3 = np.array(feature_vect_dense[2]).reshape(-1,)
similarity_simple = cos_similarity(vect1, vect3)
print('sentence1, sentence3 cosine similarity: {:.3f}'.format(similarity_simple))

vect2 = np.array(feature_vect_dense[1]).reshape(-1,)
vect3 = np.array(feature_vect_dense[2]).reshape(-1,)
similarity_simple = cos_similarity(vect2, vect3)
print('sentence2, sentence3 cosine similarity: {:.3f}'.format(similarity_simple))
# numpy.float64' can't '{0:.3f}'


sentence1, sentence3 cosine similarity: 0.465
sentence2, sentence3 cosine similarity: 0.376


사이킷런은 코사인 유사도 측정을 위해 api를 제공한다. 이를 이용해보자. 입력 파라미터는 비교 기준이 되는 문서의 피처행렬, 두번째 파라미터는 비교되는 문서의 피처행렬이다. 꼭 밀집행렬이 아니라도 가능하며 행렬 또는 배열 모두 가능하다. 앞에서처럼 굳이 변환작업이 필요하진 않다.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_simple_pair = cosine_similarity(feature_vect_simple[0], feature_vect_simple)
print(similarity_simple_pair)

[[1.         0.30396839 0.4653398 ]]


첫번째 유사도값인 1은 비교기준인 첫번째 문서자신에 대한 유사도 측정이며 두번째는 첫번째와 두번째 문서, 마지막은 첫번째와 세번째 문서 사이의 유사도 값이다. 만약 1이 거슬릴경우 비교기준 문서를 제할 수 있다.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_simple_pair = cosine_similarity(feature_vect_simple[0], feature_vect_simple[1:])
print(similarity_simple_pair)

[[0.30396839 0.4653398 ]]


### OPinion reveiw 데이터 세트를 이용한 문서 유사도 측정


#####파일리딩이 전혀 안돼서 코드로만 작성합니다....
```

from nltk.stem import WordNetLemmatizer
  import nltk
  import string

# 단어 원형 추출 함수
lemmar = WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

# 특수 문자 사전 생성: {33: None ...}
# ord(): 아스키 코드 생성
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

# 특수 문자 제거 및 단어 원형 추출
def LemNormalize(text):
    # 텍스트 소문자 변경 후 특수 문자 제거
    text_new = text.lower().translate(remove_punct_dict)
    
    # 단어 토큰화
    word_tokens = nltk.word_tokenize(text_new)
    
    # 단어 원형 추출
    return LemTokens(word_tokens)

```





```
from nltk.stem import WordNetLemmatizer
import nltk
import string

# 단어 원형 추출 함수
lemmar = WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

# 특수 문자 사전 생성: {33: None ...}
# ord(): 아스키 코드 생성
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

# 특수 문자 제거 및 단어 원형 추출
def LemNormalize(text):
    # 텍스트 소문자 변경 후 특수 문자 제거
    text_new = text.lower().translate(remove_punct_dict)
    
    # 단어 토큰화
    word_tokens = nltk.word_tokenize(text_new)
    
    # 단어 원형 추출
    return LemTokens(word_tokens)```





```
import pandas as pd
import glob, os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

path =
all_files = glob.glob(os.path.join(path, '*.data'))
filename_list = []
opinion_text = []

for file_ in all_files:
  df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')
  filename_ = file_.split('\\')[-1]
  filename = filename_.split('.')[0]
  filename_list.append(filename)
  opinion_text.appedn(df.to_string())

document_df = pd.DataFrame({'filename':'filename_list', 'opinion_text':'opinion_text'})
tfidf_vect = TfidfVectorizer((tokenizer = LemNormalize, stop_words='english',
                              ngram_range=(1,2), min_df=0.05, max_df=0.85))
feature_text = tfidf_vect.fit_transform(document_df['opinion_text'])

km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_
document_df['cluster_label'] = cluster_label```



호텔을 주제로 군집화된 문서를 이용해 특정 문서와 다른 문서간의 유사도를 알아보고자 한다. 문서를 피처벡터화 변환하고 cosine similarity를 이용해 유사도를 확인해보고자 한다.

먼저 군집화된 데이터를 추출하고 이 데이터에 해당ㅎ라는 tfidf vectorizer의 데이터를 추출하고자 한다. 호텔 군집화 데이터를 기반으로 별도의 tfidf 벡텇화를 수행하는게 아니라 바로 위에서 추출할 것이다.

데이터프레임 객체변수인 document Df 에서 먼저 호텔로 군집화된 문서의 인덱스를 추출한다. 이렇게 추출된 인덱스를 그대로 이용해 feature vectord에서 군집화된 문서의  피처 벡터를 추출한다.



```
from sklearn.metrics.pairwise import cosine_similarity

# cluster label=1 인 데이터는 호텔로 군집화된 데이터. 데이터프레임에서 이를 추출
hotel_indexes = document_df[document_df['cluster_label']==1].index
print('hotel clustered documents dataframe index: ', hotel_indexes)

# among hotel clustered datas, first data extraction, filename
comparison_docname = document_df.iloc[hotel_indexes[0]]['filename']
print('비교 기준 문서명 ', comparison_docname, '와 타 문서 유사도')

'''document df에서 추출된 index 객체를 feature vec로 입력해 호텔 군집화된 feature vect 추출
이를 이용해 호텔로 군집화된 문서 중 첫번쨰 문서와 다른 문서간의 코사인 유사도 추출''''
similariy_pair = cosine_similarity(feature_vect[hotel_indexes[0]], feature_vect[hotel_indexes])
print(similarity_pair)
```



숫자로만 표현해서는 직관적인 유사도를 알기 어려우므로 첫번째와 문서간 유사도가 높은 순으로 이를 정렬하고 시각화해보고자 함



```
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# 첫번째 문서와 타 문서간의 유사도가 큰 순으로 정렬한 인덱스를 추출하되 자기 자신은 제외
sorted_index = similarity_pair.argsort()[:,::-1]
sorted_index = sorted_inex[:,1:]

#유사도가 큰 순으로 hotel indexes를 추출해 재정렬
hotel_sorted_indexes = hotel_indexes[sorted_index.reshape(-1)]

# 유사도가 큰 순으로 재정렬하되 자기자신 제외
hotel_1_sim_value = np.sort(similarity_pair.rehsape(-1)[::-1])
hotel_1_sim_value = hotel_1_sim_value[1:]

# 유사도가 큰 순으로 정렬된 인덱스와 유사도값을 이용해 파일명과 유사도값을 막대 그래프로 시가괗
hotel_1_sim_df = pd.DataFrame()
hotel_1_sim_df['filename'] = document_df.iloc[hotel_sorted_index]['filename']
hotel_1_sim_df['similarity']=hotel_1_sim_value

sns.barplot(x='similarity', y='filename', data=hotel_1_sim_df)
plt.title(comparison_docname)```



첫번째 문서인 샌프란시스코의 베스트웨스턴 호텔화장실 리뷰와 가장 비슷한 홑문서는 room_holiday_inn_london으로 0.5정도의 코사인 유사도를 나타낸다

## 09 한글 텍스트 처리 - 네이버 영화 평점 감상분석

### 한글 NLP 처리의 어려움
한국어는 띄어쓰기와 다양한 조사로 인해 처리가 어려렵다. 특히 띄어쓰기의 경우 잘못될 경우 의미가 왜곡되며 조사가 다양하기 때문에 어렵다.

### KoNLP 소개
대표적인 한글 형태소 패키지로 형태소란 사전적으로 단어로서 의미를 갖는 최소단위를 의미한다. 형태소 분석이란 말뭉치를 이러한 형태소 어근 단위로 쪼개고 각 형태소에 품사태깅을 부착하는 작업을 일반적으로 지칭한다.

이전에는 거의 없었으며 대부분 씨나 자바 기반 패키지로 개발되었으나 konlpy는 기존의 씨나 자바기반의 형태소엔진을 파이선 래퍼 기준으로 재작성한 패키지이다. 기존의 5개 형태소 분석모델을 모두 사용가능하나 mecab의 경우 윈도우환경에선ㄴ 구동이 불가능하다.


konlpy는 공식 설치문서를 참조하는데 이는 자바와 씨 기반이기 때문에 자바가 먼저 설치되어 있어야 한다. 

jpype1을 찾아 개인 pc 환경에 맞는 것을 찾아 모듈을 설치한다3

In [ ]:
#C:\Users\chkwon\conda install c- conda-forge jpype1

아나콘다를 사용하지 않고도 pip으로도 셋업이 가능한데 요즘 버전에 맞게 파일명을 변경해야 한다.

In [ ]:

#C:\Users\chkwon> pip install --upgrade pip
#C:\Users \chkwon> pip install JPype1-0.6.3-cp36-cp36m-win_amd64.wh1

이제 자바환경설정이 필요한데 자바홈 설정이 어려운 경우 윈도우 환경설치문서의 java home 설정하기를 클릭하면 도움을 받을 수 있다.

자바홈은 압축이 풀리는 기준 jdk 폴더를 자바홈을 설정하므로 디렉토리를 jvm.dll이 들어있는 폴더로 재설정해주어야 한다.


In [2]:
pip install konlpy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 데이터 로딩

https://github.com/e9t/nsmc

테스트 데이터가 별도로 있으니 이를 이용해 평가한다.

In [13]:
import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/ESAA/OB/ratings_train.txt', sep='\t')
train_df.head(3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [14]:
# 학습 데이터세트의 0과 1라벨값 비교
train_df['label'].value_counts()


0    75173
1    74827
Name: label, dtype: int64

0과 1의 비율이 한쪽으로 치우치지 않고 균등한 분포를 나타낸다. train_df의 경우 리뷰텍스트를 갖는 document 칼럼에 null이 일부 존재하므로 이 값은 공백으로 변환한다. 문자가 아닌 숫자의 경우 단어적인 의미로 부족하므로 파이썬의 정규표현식 모듈인 re를이용해 공백으로 변환한다. 테스트 데이터세트도 동일하게 가공을 수행한다.

In [15]:
import re

train_df = train_df.fillna(' ')
# 정규 표현식을 이용해 숫자를 공백으로 변경)
train_df['document'] = train_df['document'].apply(lambda x : re.sub(r'\d+',' ', x))

# loading test data set, transform null and num to ' '
test_df = pd.read_csv('/content/drive/MyDrive/ESAA/OB/ratings_test.txt', sep='\t')
test_df = test_df.fillna(' ')
test_df['document']=test_df['document'].apply(lambda x:re.sub(r'\d+', ' ',x))

# delete id column
train_df.drop('id', axis=1, inplace=True)
test_df.drop('id', axis=1, inplace=True)

tfidf 방식으로 단어 벡터화, 먼저 각 문장을 한글형태소 분석을 통해 형태소 단어로 토큰화한다. 한글 형태소 엔진은 sns 분석에 적합한 twitter 클래스를 이용한다. twitter 객체의 morphus 메서드를 이용시 입력인자로 들어온 문장을 형태소단어 형태로 토큰화해 list 객체로 변환한다. 문장을 형태소 단어로 반환하는 토크나이저 함수 tw tokenizer를 생성한다.

In [16]:
from konlpy.tag import Twitter
twitter = Twitter()
def tw_tokenizer(text):
  # 입력인자로 들어온 텍스트를 형태소 단어로 토큰화해 리스트 형태로 반환
  tokens_ko = twitter.morphs(text)
  return tokens_ko

/usr/local/lib/python3.10/dist-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [ ]:
# tfidf model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# twitter 객체의 morphs 객체를 이요한 tokenizer 사용, ngramrange는 1,2
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

In [ ]:
#logistic regression 감성분석
lg_clf = LogisticRegression(random_state=0)

#파라미터 c 최적화를 위해 gridsearch 를 이용
params = {'C':[1,3.5,4.5, 5.5,10]}
grid_cv = GridSearchCV(lg_clf, param_grid = params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(tfidf_matrix_train, train_df['label'])
print(grid_cv.best_params_, round(grid_cv.best_score_,4))